<a href="https://colab.research.google.com/github/InowaR/robot/blob/main/radar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
from google.colab import output

field = np.zeros((16, 16))
field = np.pad(field, pad_width=1, mode='constant', constant_values=1)

def obstacle(x, y, len):
  i = y
  while i < len + y:
    field[x][i] = 1
    i += 1
  return field

obstacle(4,8,9)
obstacle(9,1,9)
obstacle(13,8,9)

field[15][8] = 2
print(field)
print("Enter \"q\" to exit")

def radar(x, y):
  l = 0    #
  m = 0    #   radar distances
  r = 0    #
  i = x    
  j = y    
  while field[i][j] != 1:
    l += 1
    i -= 1
    j -= 1
  i = x    
  j = y    
  while field[i][j] != 1:
    m += 1
    i -= 1
  i = x    
  j = y    
  while field[i][j] != 1:
    r += 1
    i -= 1
    j += 1
  print(f"Radar distances: {l, m, r}")

def play(x, y):
  i = x    
  j = y
  while True:
    try:
      button = input()
      output.clear()
      if button == "w":
        print(field)
        if field[i-1][j] == 1:
          radar(i, j)
          continue  
        field[i-1][j] = 2
        output.clear()
        print(field)
        i -= 1
        radar(i, j)
      elif button == "a":
        print(field)
        if field[i][j-1] == 1:
          radar(i, j)
          continue  
        field[i][j-1] = 2
        output.clear()
        print(field)
        j -= 1
        radar(i, j)
      elif button == "s":
        print(field)
        if field[i+1][j] == 1:
          radar(i, j)
          continue  
        field[i+1][j] = 2
        output.clear()
        print(field)
        i += 1
        radar(i, j)
      elif button == "d":
        print(field)
        if field[i][j+1] == 1:
          radar(i, j)
          continue           
        field[i][j+1] = 2
        output.clear()
        print(field)
        j += 1
        radar(i, j)
      else:
        print(field)
        radar(i, j)
        print("Exit map")
        break
    except:
      print(field)
      radar(i, j)
      continue

play(15, 8)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.